In [315]:
import os
import requests
import re
import csv
import json
import time
import html
from geojson import Feature, FeatureCollection, Point

---
# Scripts to scrape ski area data from wikipedia

---
## Get list of wikipedia url extensions for ski areas

`rawskidata.txt` is just the raw html from https://en.wikipedia.org/wiki/List_of_ski_areas_and_resorts_in_the_United_States

In [4]:
skiareas = []
with open("rawskidata.txt") as f:
    for line in f:
        match = re.search('<li>', line)
        if(match):
            match = re.search('href\S+', line)
            if(match):
                skiareas.append(match.group(0)[6:-1])

---

### testing cells

In [364]:
#viewcount_2020_2014 = viewcount_all[0:6]
#len(viewcount_2020_2014)

6

In [161]:
URL = 'https://en.wikipedia.org' + skiareas[81]
data = requests.get(URL, stream=True)
#     college_sb_data.append(pd.Series(name=str(yr)))
for line in data.iter_lines():
    match = re.search('Vertical</th><td>[\D]*(.[\d,]*)', str(line))
    if(match):
        if(len(match.group(1)) < 2):
            match = re.search('Vertical</th><td>(.*)', str(line))
            match2 = re.search('</span>(.[\d,]*)', match.group(0))
            vert_value = match2.group(1).replace(',', '')
            if(len(match2.group(1)) >= 2):
                print(vert_value)
            elif(len(match2.group(1)) < 2):
                match2 = re.search('&#160; &#160;(.[\d,]*)', match.group(0))
                vert_value = match2.group(1).replace(',', '')
                print(vert_value)
        elif(len(match.group(1)) >= 2):
            vert_value = match.group(1).replace(',', '')
            print(vert_value)
        else:
            print("could not find vertical!")
            skiareas_novert.append(skiareas[num])

Vertical</th><td><sup id="cite_ref-1


---
## Go through list of ski areas and get verts

yummy unoptimized spaghetti code yum yum

In [203]:
skiareas_novert = []
verts = []
verts_all = []
time_start = time.perf_counter()
time_elapsed = 0
for num in range(len(skiareas)):
    URL = 'https://en.wikipedia.org' + skiareas[num]
    data = requests.get(URL, stream=True)
    vert_value = -1 # null value
    top_value = -1
    base_value = -1
    for line in data.iter_lines():
        match = re.search('Vertical</th><td>(.[\d,]*)', str(line))
        if(match):
            if(len(match.group(1)) < 2): # handle inconsistent formatting -- TODO tidy this up
                match = re.search('Vertical</th><td>(.*)', str(line))
                match2 = re.search('</span>(.[\d,]*)', match.group(0))
                if(match2):
                    if(len(match2.group(1)) >= 2):
                        vert_value = match2.group(1).replace(',', '')
                        #print(vert_value)
                    elif(len(match2.group(1)) < 2):
                        match2 = re.search('&#160; &#160;(.[\d,]*)', match.group(0))
                        if(match2):
                            if(len(match2.group(1)) >= 2):
                                vert_value = match2.group(1).replace(',', '')
                                #print(vert_value)
                    if(vert_value == -1):
                        match2 = re.search('</a></sup>(.[\d,]*)', match.group(0))
                        if(match2):
                            if(len(match2.group(1)) >= 2):
                                vert_value = match2.group(1).replace(',', '')
                                #print(vert_value)
            elif(len(match.group(1)) >= 2): # for normal formatting
                vert_value = match.group(1).replace(',', '')
                #print(vert_value)
                
        # if vertical is not given directly on page, find vert from top and bottom elevation
        matchTop = re.search('Top[\s&#160;]*elevation</th><td>(.[\d,]*)', str(line))
        matchBase = re.search('Base[\s&#160;]*elevation</th><td>(.[\d,]*)', str(line))    
        if(matchTop and matchBase):
            if(len(matchTop.group(1)) < 2):
                matchTop = re.search('Top[\s&#160;]*elevation</th><td>(.*)', str(line))
                matchTop2 = re.search('</span>(.[\d,]*)', match.group(0))
                if(matchTop2):
                    if(len(matchTop2.group(1)) >= 2):
                        top_value = matchTop2.group(1).replace(',', '')
                    elif(len(matchTop2.group(1)) < 2):
                        matchTop2 = re.search('&#160; &#160;(.[\d,]*)', matchTop.group(0))
                        if(matchTop2):
                            if(len(matchTop2.group(1)) >= 2):
                                top_value = matchTop2.group(1).replace(',', '')
                    if(top_value == -1):
                        matchTop2 = re.search('</a></sup>(.[\d,]*)', matchTop.group(0))
                        if(matchTop2):
                            if(len(matchTop2.group(1)) >= 2):
                                top_value = matchTop2.group(1).replace(',', '')
                    if(top_value == -1):
                        matchTop2 = re.search('<span\sstyle="white-space:nowrap">(.[\d,]*)', matchTop.group(0))
                        if(matchTop2):
                            if(len(matchTop2.group(1)) >= 2):
                                top_value = matchTop2.group(1).replace(',', '')
            elif(len(matchTop.group(1)) >= 2):
                top_value = matchTop.group(1).replace(',', '')
                
            if(len(matchBase.group(1)) < 2):
                #print("bottom value was formatted incorrectly. attempting to fix...") #DEBUG
                matchBase = re.search('Base[\s&#160;]*elevation</th><td>(.*)', str(line))
                matchBase2 = re.search('</span>(.[\d,]*)', matchBase.group(0))
                if(matchBase2):
                    if(len(matchBase2.group(1)) >= 2):
                        base_value = matchBase2.group(1).replace(',', '')
                    elif(len(matchBase2.group(1)) < 2):
                        matchBase2 = re.search('&#160; &#160;(.[\d,]*)', matchBase.group(0))
                        if(matchBase2):
                            if(len(matchBase2.group(1)) >= 2):
                                base_value = matchBase2.group(1).replace(',', '')
                    if(base_value == -1):
                        matchBase2 = re.search('</a></sup>(.[\d,]*)', matchBase.group(0))
                        if(matchBase2):
                            if(len(matchBase2.group(1)) >= 2):
                                base_value = matchBase2.group(1).replace(',', '')
            elif(len(matchBase.group(1)) >= 2):
                base_value = matchBase.group(1).replace(',', '')
            
            if(top_value != -1 and base_value != -1):
                vert_value = int(top_value) - int(base_value)
                #print(skiareas_novert[num], "vertical:", vert_value)
    if(vert_value == -1):
        skiareas_novert.append(skiareas[num])
        verts_all.append("null")
    else:
        verts.append(vert_value)
        verts_all.append(vert_value)
    time_elapsed = time.perf_counter() - time_start
    if((num + 1) % 30 == 0):
        print("\n", num + 1, "areas processed.", int(((num + 1) / len(skiareas)) * 100), "% done")
        print(int(time_elapsed), "seconds elapsed.")
        eta = int((time_elapsed / num + 1) * (len(skiareas) - num + 1))
        if(eta > 59):
            eta_min = int(eta / 60)
            eta_sec = int(eta % eta_min)
            print("ETA:", eta_min, "minutes and", eta_sec, "seconds.")
        else:
            print("ETA:", eta, "seconds.")
        #print("Current list of real verts with indices:", verts, "\n\n")

print("453 areas processed in", int(time_elapsed), "seconds")
print("Could not find vert for", len(verts_all) - len(verts), "areas.")


 30 areas processed. 6 % done
6 seconds elapsed.
ETA: 8 minutes and 2 seconds.

 60 areas processed. 13 % done
11 seconds elapsed.
ETA: 7 minutes and 6 seconds.

 90 areas processed. 19 % done
17 seconds elapsed.
ETA: 7 minutes and 4 seconds.

 120 areas processed. 26 % done
30 seconds elapsed.
ETA: 6 minutes and 5 seconds.

 150 areas processed. 33 % done
39 seconds elapsed.
ETA: 6 minutes and 0 seconds.

 180 areas processed. 39 % done
50 seconds elapsed.
ETA: 5 minutes and 2 seconds.

 210 areas processed. 46 % done
62 seconds elapsed.
ETA: 5 minutes and 3 seconds.

 240 areas processed. 52 % done
71 seconds elapsed.
ETA: 4 minutes and 3 seconds.

 270 areas processed. 59 % done
86 seconds elapsed.
ETA: 4 minutes and 0 seconds.

 300 areas processed. 66 % done
93 seconds elapsed.
ETA: 3 minutes and 2 seconds.

 330 areas processed. 72 % done
99 seconds elapsed.
ETA: 2 minutes and 0 seconds.

 360 areas processed. 79 % done
105 seconds elapsed.
ETA: 2 minutes and 1 seconds.

 390 ar

---
## Use wikimedia API to get pageviews

requests return null for 2014 and earlier, either pageviews aren't archived or i am no longer allowed to access the wikimedia api cause i made 20x the max calls

In [210]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36/Corin Zarkowski coreyz206@gmail.com'}
viewcount_all = []
time_start = time.perf_counter()
current_index = 0
time_elapsed = 0
for year in reversed(range(2011, 2021)):
    viewcount_all.append([])
    for num in range(len(skiareas)):
        viewcount = 0
        URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/' + skiareas[num][6:] + '/monthly/' + str(year) + '010100/' + str(year) + '123100'
        data = requests.get(url = URL, headers = headers)
        if data.status_code == 200:
            data_json = data.json()
            for month in data_json['items']:
                viewcount += month['views']
            viewcount_all[current_index].append(viewcount)
        else:
            print("request denied!!!!!! :((((((( on", skiareas[num], "year:", year, "appending null\n")
            viewcount_all[current_index].append('null')
        
        time_elapsed = time.perf_counter() - time_start
        if((num + 1) % 60 == 0):
            print("\n", (num + 1), "areas processed.", int((num + 1) / (len(skiareas)) * 100), "% done")
            print("year:", year)
            print(int(time_elapsed), "seconds elapsed.")
            print(viewcount_all)
    current_index += 1;

print("453 areas over 10 years processed in", int(time_elapsed), "seconds")

request denied!!!!!! :((((((( on /w/index.php?title=Hermon_Mountain_(ski_area)&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Jefferson_Ski_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Quoggy_Jo&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Greylock_Ski_Club&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Prospect_Ski_Tow&amp;action=edit&amp;redlink=1 year: 2020 appending null


 60 areas processed. 13 % done
year: 2020
7 seconds elapsed.
[[7258, 2923, 4329, 4156, 1705, 2536, 347, 5292, 2614, 1354, 'null', 1173, 2093, 239, 'null', 1442, 'null', 11261, 5404, 31438, 1113, 1750, 7261, 7790, 3710, 3807, 14499, 1326, 7611, 'null', 5862, 965, 6056, 2277, 8834, 1575, 11753, 3001, 7762, 10932, 6703, 10984, 6975, 2537, 22202, 

request denied!!!!!! :((((((( on /w/index.php?title=Garland_Resort&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Hickory_Hills_Ski_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=The_Homestead_(resort)&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mt._Holiday&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mt._Holly_Ski_and_Snowboard_Resort&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mt._McSauba_Recreation_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mt._Zion_Ski_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Norway_Mountain_Ski_and_Snowboa

request denied!!!!!! :((((((( on /w/index.php?title=Chapman_Hill_Ski_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Cranor_Ski_Area&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Lake_City_Ski_Hill&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Loveland_Valley&amp;action=edit&amp;redlink=1 year: 2020 appending null


 300 areas processed. 66 % done
year: 2020
79 seconds elapsed.
[[7258, 2923, 4329, 4156, 1705, 2536, 347, 5292, 2614, 1354, 'null', 1173, 2093, 239, 'null', 1442, 'null', 11261, 5404, 31438, 1113, 1750, 7261, 7790, 3710, 3807, 14499, 1326, 7611, 'null', 5862, 965, 6056, 2277, 8834, 1575, 11753, 3001, 7762, 10932, 6703, 10984, 6975, 2537, 22202, 5273, 19249, 4518, 'null', 6923, 4708, 5287, 822, 52704, 10102, 3112, 10079, 3340, 4429, 313, 13196, 7384, 8953, 19946, 40512, 14674, 722, 4781

request denied!!!!!! :((((((( on /w/index.php?title=Elko_Snobowl_Ski_and_Bike_Park&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Sahalie_Ski_Club&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Sitzmark_Lifts&amp;action=edit&amp;redlink=1 year: 2020 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Hermon_Mountain_(ski_area)&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Jefferson_Ski_Area&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Quoggy_Jo&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Greylock_Ski_Club&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Prospect_Ski_Tow&amp;action=ed

request denied!!!!!! :((((((( on /w/index.php?title=Eagle_Rock_Resort&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Ski_Denton&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Ski_Sawmill&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Cloudmont_Ski_%26_Golf_Resort&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Sapphire_Valley&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Ski_Beech&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Sugar_Mountain_(ski_area)&amp;action=edit&amp;redlink=1 year: 2019 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Mount_Aggie_Ski_Slope&amp;action=edit&amp;redlink=1 year: 2019

...

request denied!!!!!! :((((((( on /wiki/Brownsville,_Vermont year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Bolton_Valley_Resort year: 2012 appending null


 60 areas processed. 13 % done
year: 2012
1028 seconds elapsed.
[[7258, 2923, 4329, 4156, 1705, 2536, 347, 5292, 2614, 1354, 'null', 1173, 2093, 239, 'null', 1442, 'null', 11261, 5404, 31438, 1113, 1750, 7261, 7790, 3710, 3807, 14499, 1326, 7611, 'null', 5862, 965, 6056, 2277, 8834, 1575, 11753, 3001, 7762, 10932, 6703, 10984, 6975, 2537, 22202, 5273, 19249, 4518, 'null', 6923, 4708, 5287, 822, 52704, 10102, 3112, 10079, 3340, 4429, 313, 13196, 7384, 8953, 19946, 40512, 14674, 722, 4781, 32311, 575, 22238, 5638, 1680, 588, 5197, 22890, 13950, 20574, 8003, 17038, 41026, 5120, 4476, 38866, 'null', 9937, 'null', 'null', 8819, 'null', 7256, 'null', 14464, 582, 'null', 11440, 'null', 'null', 15134, 'null', 'null', 'null', 1981, 'null', 4892, 3544, 'null', 'null', 'null', 'null', 'null', 5102, 3273, 'null', 3690, 'null'

request denied!!!!!! :((((((( on /wiki/Bromley_Mountain year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Burke_Mountain_(Vermont) year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Skiing_Cochrans year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Jay_Peak_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Killington_Ski_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mad_River_Glen year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Magic_Mountain,_Vermont year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Middlebury_College_Snow_Bowl year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mount_Snow year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Northeast_Slopes year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Okemo_Mountain year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Pico_Mountain year: 2012 app

request denied!!!!!! :((((((( on /wiki/Blue_Mountain_Ski_Area year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Boyce_Park year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Camelback_Ski_Area year: 2012 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Eagle_Rock_Resort&amp;action=edit&amp;redlink=1 year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Elk_Mountain_Ski_Area year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Hidden_Valley_Four_Seasons_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Jack_Frost_Ski_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Laurel_Mountain_Ski_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Liberty_Mountain_Resort year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mountain_View_at_Edinboro year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Seven_Springs_Mountain_Resort ye

request denied!!!!!! :((((((( on /w/index.php?title=Challenge_Mountain&amp;action=edit&amp;redlink=1 year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Crystal_Mountain_(Michigan) year: 2012 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Garland_Resort&amp;action=edit&amp;redlink=1 year: 2012 appending null

request denied!!!!!! :((((((( on /w/index.php?title=Hickory_Hills_Ski_Area&amp;action=edit&amp;redlink=1 year: 2012 appending null

request denied!!!!!! :((((((( on /w/index.php?title=The_Homestead_(resort)&amp;action=edit&amp;redlink=1 year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Indianhead_Mountain year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Marquette_Mountain year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mont_Ripley year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mount_Bohemia year: 2012 appending null

request denied!!!!!! :((((((( on /wiki/Mt._Brighton year: 201

KeyboardInterrupt: 

---
## Pull names and coordinates of ski areas

In [339]:
names_all = []
lat_all = []
lng_all = []

time_start = time.perf_counter()
time_elapsed = 0

for num in range(len(skiareas)):
    URL = 'https://en.wikipedia.org' + skiareas[num]
    data = requests.get(URL, stream=True)
    name = None
    coords = None
    lat = None
    lng = None
    for line in data.iter_lines():
        if(line):
            line = str(line)[2:-1] # account for extra byte class chars
            line = html.unescape(line)

            match = re.search('<title>(.*)</title>', line)
            if(match):
                if(len(match.group(1)) > 1):
                    name = match.group(1)[:-12] # remove '- Wikipedia' from name
            
            match = re.search('<span class="geo">([\d\.-]*); ([\d\.-]*)', line)
            if(match):
                lat = match.group(1)
                lng = match.group(2)
                
            
    if name:
        names_all.append(name)
    else:
        names_all.append('null')
        print('Could not find name for', skiareas[num], "!")
    if lat and lng:
        lat_all.append(lat)
        lng_all.append(lng)
    else:
        lat_all.append('null')
        lng_all.append('null')
        print('Could not find coords for', skiareas[num], "!")
    
    time_elapsed = time.perf_counter() - time_start
    if((num + 1) % 40 == 0):
        print("\n", (num + 1), "areas processed.", int((num + 1) / (len(skiareas)) * 100), "% done")
        print(int(time_elapsed), "seconds elapsed.")

print("453 areas processed in", int(time_elapsed), "seconds")
print(names_all)
print(lat_all, lng_all)

Could not find coords for /w/index.php?title=Hermon_Mountain_(ski_area)&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Mount_Jefferson_Ski_Area&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Quoggy_Jo&amp;action=edit&amp;redlink=1 !
Could not find coords for /wiki/Bousquet_Ski_Area !
Could not find coords for /w/index.php?title=Mount_Greylock_Ski_Club&amp;action=edit&amp;redlink=1 !

 40 areas processed. 8 % done
9 seconds elapsed.
Could not find coords for /w/index.php?title=Mount_Prospect_Ski_Tow&amp;action=edit&amp;redlink=1 !

 80 areas processed. 17 % done
16 seconds elapsed.
Could not find coords for /w/index.php?title=Beartown_Ski_Area&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Big_Tupper_Ski_Area&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Brantling_Ski_Slopes&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Buffalo_Ski_Club&

Could not find coords for /w/index.php?title=Camp_10&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Christie_Mountain&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Christmas_Mountain_Village&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Crystal_Ridge&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Fox_Hill_Ski_Area&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Heiliger_Huegel_Ski_Club&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Kewaunee_County_Winter_Park_Ski_Hill&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Keyes_Peak&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Little_Switzerland_Ski_Area&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/index.php?title=Mont_Du_Lac&amp;action=edit&amp;redlink=1 !
Could not find coords for /w/ind

---

### Aggregate individual lists into one BIG list and write to csv for safekeeping

In [365]:
skiarea_formatted = []
viewcount_2020_2015 = viewcount_all[0:6]

for num in range(len(skiareas)):
    skiarea_formatted.append([names_all[num], lat_all[num], lng_all[num], verts_all[num]])
    for year in reversed(viewcount_2020_2015):
        skiarea_formatted[num].append(year[num])

skiarea_formatted

[['Mohawk Mountain Ski Area',
  '41.83675',
  '-73.31342',
  '650',
  3081,
  8594,
  7134,
  6528,
  9250,
  7258],
 ['Mount Southington',
  '41.58235',
  '-72.92587',
  425,
  1174,
  2973,
  3646,
  2689,
  2784,
  2923],
 ['Powder Ridge Ski Area',
  '41.502006',
  '-72.737172',
  '550',
  1343,
  4028,
  4819,
  4257,
  4039,
  4329],
 ['Ski Sundown',
  '41.88407',
  '-72.94523',
  '625',
  1317,
  3787,
  5451,
  5018,
  4306,
  4156],
 ['Baker Mountain (ski area)',
  '45.08472',
  '-69.90556',
  227,
  1160,
  2097,
  2138,
  1992,
  1658,
  1705],
 ['Big Rock (ski resort)',
  '46.52222',
  '-67.82778',
  920,
  1381,
  3392,
  3750,
  2660,
  2660,
  2536],
 ['Big Squaw Mountain Resort',
  '45.50417',
  '-69.70556',
  1620,
  342,
  627,
  508,
  427,
  467,
  347],
 ['Black Mountain of Maine',
  '44.58000',
  '-70.61750',
  '1380',
  1480,
  3578,
  3430,
  3155,
  3809,
  5292],
 ['Camden Snow Bowl',
  '44.217216',
  '-69.134667',
  845,
  1140,
  2455,
  3293,
  2983,
  2652,

In [366]:
fields = ['name', 'lat', 'lng', 'vert', 'viewcount2020', 
          'viewcount2019', 'viewcount2018', 'viewcount2017', 
          'viewcount2016', 'viewcount2015']
with open('skiareas_v2.csv', 'w') as f: 
      
    write = csv.writer(f) 
      
    write.writerow(fields) 
    write.writerows(skiarea_formatted) 

In [19]:
cityjsonsorted = []
citynum = 0
for city in cityjson:
    cityjsonsorted.append([])
    cityjsonsorted[citynum].append(city['city'])
    cityjsonsorted[citynum].append(city['growth_from_2000_to_2013'])
    cityjsonsorted[citynum].append(city['latitude'])
    cityjsonsorted[citynum].append(city['longitude'])
    cityjsonsorted[citynum].append(city['population'])
    cityjsonsorted[citynum].append(city['rank'])
    cityjsonsorted[citynum].append(city['state'])
    citynum += 1

In [25]:
new_fields = ['city_name', 'growth_2000_2013', 'lat', 'lng', 'pop', 'rank_size', 'state']
with open('cities_sorted.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(new_fields)
    write.writerows(cityjsonsorted)

In [369]:
features = []
for area in skiarea_formatted:
    
    if(area[1] != 'null' and area[2] != 'null'):
        latitude, longitude = map(float, (area[1], area[2]))
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'name': area[0],
                    'vertical': area[3],
                    'pageviews': {
                        '2020': area[4],
                        '2019': area[5],
                        '2018': area[6],
                        '2017': area[7],
                        '2016': area[8],
                        '2015': area[9],
                    }
                }
            )
        )

collection = FeatureCollection(features)
with open("ski_geojson_v2.json", "w") as f:
    f.write('%s' % collection)

In [231]:
with open('viewcounts_unformatted_2020_2015.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(viewcount_filtered)

In [235]:
with open('verts_unformatted.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(verts_all)